## Introduction


Build an image classifier using the VGG16 pre-trained model, and evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model.

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:


!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip


--2025-01-19 18:39:09--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  39.6MB/s    in 6.5s    

2025-01-19 18:39:15 (38.5 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [ ]:

import zipfile
import os

# Path to the zip file
zip_path = 'concrete_data_week4.zip'

# Folder to extract to
extraction_path = ''

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

# Verify the contents
extracted_files = os.listdir(extraction_path)
print(extracted_files)

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [19]:
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [3]:
# Assuming you have your training and validation data directories defined as:
train_dir = 'concrete_data_week4/train'
validation_dir = 'concrete_data_week4/valid'

# Define the ImageDataGenerators with batch size
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Apply VGG16 preprocessing
    # ... other data augmentation parameters if needed
).flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize images to VGG16 input size
    batch_size=100,
    class_mode='categorical'  # Assuming you have more than 2 classes
)

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
).flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=100,
    class_mode='categorical'
)

# ... (Rest of your model building and training code)

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [9]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Assuming you have your training and validation data directories defined as:
train_dir = 'concrete_data_week4/train'
validation_dir = 'concrete_data_week4/valid'

# Create an ImageDataGenerator for the training set
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Apply VGG16 preprocessing
    # You can add other data augmentation options here if needed
    # such as rotation, shear, zoom, etc.
    # Example: rotation_range=20, width_shift_range=0.2, ...
)

# Create an ImageDataGenerator for the validation set
validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input  # Apply VGG16 preprocessing
)

# Create data generators for training and validation with batch size 100
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize images to VGG16 input size
    batch_size=100,          # Set batch size to 100
    class_mode='categorical'  # Use 'binary' if you have 2 classes
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),  # Resize images to VGG16 input size
    batch_size=100,          # Set batch size to 100
    class_mode='categorical'  # Use 'binary' if you have 2 classes
)

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [10]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Create the base model from VGG16 (include_top=False to exclude the classifier)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers (prevent them from being updated during training)
for layer in base_model.layers:
    layer.trainable = False

# Create a sequential model
model = Sequential()

# Add the VGG16 base model
model.add(base_model)

# Add a flattening layer to convert the output of VGG16 to a 1D vector
model.add(Flatten())

# Add a dense layer for classification (adjust units and activation as needed)
model.add(Dense(units=2, activation='softmax'))  # Example for binary classification

# Print the model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │          50,178 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,764,866 (56.32 MB)

 Trainable params: 50,178 (196.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [27]:


# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [28]:
# ... (previous code to create and compile the model)

# Fit the model using the ImageDataGenerators
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10  # Adjust the number of epochs as needed
)
model.save('vgg16_model.h5')
print("VGG16 model saved as vgg16_model.h5")

Epoch 1/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 829s 3s/step - accuracy: 0.9536 - loss: 2.8032 - val_accuracy: 0.9967 - val_loss: 0.0138
Epoch 2/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 563us/step - accuracy: 0.9800 - loss: 0.0839 - val_accuracy: 1.0000 - val_loss: 2.8014e-05
Epoch 3/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 699s 2s/step - accuracy: 0.9967 - loss: 0.0108 - val_accuracy: 0.9965 - val_loss: 0.0124
Epoch 4/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 224us/step - accuracy: 0.9900 - loss: 0.0108 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 697s 2s/step - accuracy: 0.9967 - loss: 0.0113 - val_accuracy: 0.9959 - val_loss: 0.0155
Epoch 6/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 227us/step - accuracy: 1.0000 - loss: 3.7029e-04 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 695s 2s/step - accuracy: 0.9980 - loss: 0.0080 - val_accuracy: 0.9980 - val_loss: 0.0082
Epoch 8/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 227us/step - accuracy: 1.0000 - los

VGG16 model saved as vgg16_model.h5


<a id="item43"></a>

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model.
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

Creating the ResNet50 model

In [21]:


# Build the ResNet50 model
resnet_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in resnet_base_model.layers:
    layer.trainable = False

# Add custom layers
resnet_model = Sequential([
    resnet_base_model,
    Flatten(),
    Dense(256, activation='relu'),  # Add a dense layer
    Dense(2, activation='softmax')  # Adjust for your number of classes
])

# Compile the model
resnet_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model (Use your training and validation generators)
resnet_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10  # Adjust as necessary
)

# Save the ResNet50 model
resnet_model.save('resnet50_model.h5')
print("ResNet50 model saved as 'resnet50_model.h5'")


Epoch 1/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9615 - loss: 1.4677

C:\Users\mahar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


300/300 ━━━━━━━━━━━━━━━━━━━━ 850s 3s/step - accuracy: 0.9616 - loss: 1.4640 - val_accuracy: 0.9965 - val_loss: 0.0261
Epoch 2/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 588us/step - accuracy: 1.0000 - loss: 3.7742e-05 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10


C:\Users\mahar\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


300/300 ━━━━━━━━━━━━━━━━━━━━ 694s 2s/step - accuracy: 0.9959 - loss: 0.0156 - val_accuracy: 0.9952 - val_loss: 0.0141
Epoch 4/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 213us/step - accuracy: 0.9900 - loss: 0.0171 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 690s 2s/step - accuracy: 0.9966 - loss: 0.0150 - val_accuracy: 0.9975 - val_loss: 0.0106
Epoch 6/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 218us/step - accuracy: 0.9900 - loss: 0.0199 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 690s 2s/step - accuracy: 0.9967 - loss: 0.0134 - val_accuracy: 0.9966 - val_loss: 0.0105
Epoch 8/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 239us/step - accuracy: 1.0000 - loss: 0.0023 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 9/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 689s 2s/step - accuracy: 0.9977 - loss: 0.0085 - val_accuracy: 0.9985 - val_loss: 0.0058
Epoch 10/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 209us/step - accuracy: 1.0000 - loss: 0.0025 - va

ResNet50 model saved as 'resnet50_model.h5'


In [29]:
# Import necessary libraries
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Test directory path
test_dir = 'concrete_data_week4/test'

# Create a test ImageDataGenerator
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),  # Resize to match input size
    batch_size=100,          # Use the same batch size
    class_mode='categorical',
    shuffle=False            # Ensure order consistency
)

# Load the saved models
vgg16_model = load_model('vgg16_model.h5')
resnet50_model = load_model('resnet50_model.h5')

# Evaluate VGG16 model
vgg16_performance = vgg16_model.evaluate(test_generator, verbose=1)
print(f"VGG16 Model Performance:\n  Loss: {vgg16_performance[0]}\n  Accuracy: {vgg16_performance[1]}")

# Evaluate ResNet50 model
resnet50_performance = resnet50_model.evaluate(test_generator, verbose=1)
print(f"ResNet50 Model Performance:\n  Loss: {resnet50_performance[0]}\n  Accuracy: {resnet50_performance[1]}")


Found 500 images belonging to 2 classes.


5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 1.0000 - loss: 1.4353e-04
VGG16 Model Performance:
  Loss: 9.680172661319375e-05
  Accuracy: 1.0
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 1.0000 - loss: 8.3833e-04
ResNet50 Model Performance:
  Loss: 0.0011117341928184032
  Accuracy: 1.0


5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 1.0000 - loss: 8.3833e-04
ResNet50 Model - Loss: 0.0011, Accuracy: 1.0000


<a id="item44"></a>

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li>
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [31]:
# ... (previous code to load the model and create test_generator)

# Make predictions using the VGG16 model
predictions = model.predict(test_generator)

# Get the predicted class labels (assuming you have a binary classification problem)
predicted_classes = (predictions > 0.5).astype(int)  # Adjust threshold if needed

# Print the predicted classes for the first 5 images
print(predicted_classes[:5])

5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]]


In [32]:
# ... (previous code to load the model and create test_generator)

# Make predictions using the VGG16 model
predictions = model.predict(test_generator)

# Get the predicted class labels (assuming binary classification)
predicted_classes = (predictions > 0.5).astype(int)

# Print predictions for the first 5 images
for prediction in predicted_classes[:5]:
    if prediction[0] == 0:  # Assuming class 0 is "Negative"
        print("Negative")
    else:  # Assuming class 1 is "Positive"
        print("Positive")

5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step
Positive
Positive
Positive
Positive
Positive
